# Logistic Regression

In [179]:


from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, recall_score

def input_mlflow(cv_r_v,increment_number,X_train, y_train,pipeline,run_name,experiment_id):
    

    for wi in range(len(cv_r_v)):
        
        #====================skema===================
        cols_spec = []
        data_map = {
                'int64': 'integer',
                'float64': 'double',
                'bool': 'boolean',
                'str': 'string',
                "date": 'datetime'
            }

        for name, dtype in X_train.dtypes.to_dict().items():
            cols_spec.append(ColSpec(name=name, type=data_map[str(dtype)]))
        input_schema = Schema(inputs=cols_spec)
        output_schema = Schema([ColSpec(name="label", type="double")])
        #parameter = ParamSpec(name="model_name", dtype="string", default="model1")
        #param_schema = ParamSchema(params=[parameter])
        model_signature = ModelSignature(inputs=input_schema, outputs=output_schema)#, params=param_schema)
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        model_signature = infer_signature(X_train, y_train)#, params={"model_name": "model1"})
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        
        
        
        
        #====================buat run baru===================
        run_name_with_increment = f"{run_name}__{increment_number}"
            # Membuka run MLflow
        with mlflow.start_run(run_name=run_name_with_increment, experiment_id=experiment_id) as run:
            # Mendapatkan run_id
            run_id = mlflow.active_run().info.run_id
        increment_number=increment_number+1
        print("run_id:", run_id)

        
        
        #====================buat parameter model dan metric===================

        # Definisikan grid parameter untuk dicari
        param_grid = cv_r_v[wi]
        
        #====================model===================

        # Inisialisasi GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='accuracy')

        # Lakukan pencarian grid
        grid_search.fit(X_train, y_train)
        
            
        #====================parameter===================
        pipe=grid_search.best_estimator_
        
        
        first_step_name = list(pipe.named_steps.keys())[0:len(pipe)]
        

        for i in range(len(pipe)):
            # Mendapatkan parameter dari langkah 'sca'
            nama=first_step_name[i]
            sca_params = pipe.get_params()[nama]

                # Membuka run MLflow
            with mlflow.start_run( experiment_id=experiment_id,run_id=run_id) as run:

                # Log parameter secara otomatis menggunakan loop
                for param_name, param_value in sca_params.get_params().items():
                    param_name=nama+'__'+param_name
                    #print(param_name,param_value)
                    mlflow.log_param(param_name, param_value)
                    
                    
                #====================metric===================
                #matric
                hasil_test=grid_search.predict(X_test)
 
                
                
                
                                # Menghitung akurasi
                accuracy = accuracy_score(y_test, hasil_test)

                # Menghitung F1 score
                f1 = f1_score(y_test, hasil_test,average='macro')

                # Menghitung recall
                recall = recall_score(y_test, hasil_test,average='macro')

                
                
                                # Menghitung precision
                precision = precision_score(y_test, hasil_test,average='macro')


   
                

                # log model 
                mlflow.sklearn.log_model(sk_model=grid_search, artifact_path="grid_search__"+str(increment_number-1),signature=model_signature)

                metrics = {
                    'mean_test_score':pd.DataFrame(grid_search.cv_results_)['mean_test_score'].values[0],
                    "accuracy": accuracy,
                    "f1": f1,
                    "recall": recall,
                    "precision": precision,
         
                }

                mlflow.log_metrics(metrics)
                
                
        mlflow.end_run()
    print('selesai')

        
    return increment_number


       

def ambil_best(grid_search,n):
    cv_r=grid_search.copy()
    cv_r_v=cv_r.sort_values(['rank_test_score'])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v

In [214]:
import mlflow.sklearn
from sklearn.metrics import classification_report, confusion_matrix


def test(model_paths,X_test2,y_test):
    ii=1
    for i, model_path in enumerate(model_paths, start=1):
        print(f'\n\n================================| model {ii} |==========================================')
        
        model = mlflow.sklearn.load_model(model_path)
        y_pred = model.predict(X_test2)
        print(f"Classification Report for model {model_path[-2:]}:")
        print(classification_report(y_test, y_pred))
        print(f"Confusion Matrix for model {model_path[-2:]}:\n", confusion_matrix(y_test, y_pred), '\n\n')
        
        ii=ii+1

## Imports

In [181]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats



import seaborn as sns
import statsmodels.api as sm
import pandas as pd



from scipy.stats import shapiro,normaltest,kstest,jarque_bera
import pingouin as pg
from pingouin import kruskal


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score


import mlflow

from mlflow.models.signature import ModelSignature
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema
from mlflow.types.schema import ParamSchema
from mlflow.types.schema import ParamSpec
from mlflow.types.schema import ColSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from typing import Tuple
from sklearn import set_config
set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


In [182]:
df = pd.read_csv('data/data_csv/data_bersih.csv')

In [183]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,virginica
145,6.3,2.5,5.0,1.9,virginica
146,6.5,3.0,5.2,2.0,virginica
147,6.2,3.4,5.4,2.3,virginica


In [184]:
X = df.drop('species',axis=1)
y = df['species']

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=141)

In [186]:
y_train.value_counts()

setosa        36
versicolor    35
virginica     33
Name: species, dtype: int64

In [187]:
y_test.value_counts()

virginica     17
versicolor    15
setosa        13
Name: species, dtype: int64

In [188]:

scatler = StandardScaler()
log_model = LogisticRegression(max_iter=10000)

In [189]:
opera=[('StandardScaler',scatler),('LogisticRegression',log_model)]
pipeline=Pipeline(opera)

## Logistic Regression Model  try all

In [190]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pipeline

Pipeline(steps=[('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(max_iter=10000))])

## param_grid_lbfgs

In [191]:
param_grid_lbfgs = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['lbfgs'],
    'LogisticRegression__penalty': ['l2', 'none']  # Perbaikan penulisan untuk 'none'
}


In [192]:
grid_search = GridSearchCV(pipeline, param_grid_lbfgs , cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['lbfgs'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [193]:
df_1 = pd.DataFrame(grid_search.cv_results_)
df_1.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_1 = df_1.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_1.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.016382,0.002580,0.003334,0.001484,2.50075,l2,lbfgs,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
21,0.009607,0.002523,0.001350,0.001758,5.0005,l2,lbfgs,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.952381,0.952381,0.904762,0.95,0.951905,0.030132,5
6,0.010433,0.002835,0.001206,0.002413,0.001,none,lbfgs,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.904762,0.952381,0.904762,0.95,0.942381,0.035520,8
5,0.020920,0.003305,0.001359,0.001313,0.001,none,lbfgs,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.90,0.932381,0.024541,14
7,0.008576,0.005043,0.002003,0.004006,0.001,none,lbfgs,5.0005,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.904762,0.952381,0.809524,0.857143,0.90,0.884762,0.048225,19
12,0.007931,0.000979,0.001590,0.003180,2.50075,l2,lbfgs,5.0005,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,0.809524,0.857143,0.90,0.875238,0.037423,27
8,0.012115,0.002321,0.005196,0.001623,0.001,none,lbfgs,7.50025,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.809524,0.857143,0.80,0.817143,0.036341,28
1,0.012754,0.003553,0.001520,0.001355,0.001,l2,lbfgs,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,37
9,0.010187,0.001610,0.003078,0.002241,0.001,none,lbfgs,10.0,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.809524,0.761905,0.80,0.798095,0.035328,41
0,0.013270,0.004057,0.000939,0.001878,0.001,l2,lbfgs,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.714286,0.666667,0.619048,0.761905,0.85,0.722381,0.079619,50


## param_grid_newton-cg 

In [194]:
param_grid_newton = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['newton-cg'],
    'LogisticRegression__penalty': ['l2', 'none']  # Perbaikan penulisan untuk 'none'
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_newton , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['newton-cg'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [195]:
df_2 = pd.DataFrame(grid_search.cv_results_)
df_2.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_2 = df_2.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_2.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,0.011030,0.003367,0.000684,0.001368,0.001,none,newton-cg,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
11,0.009684,0.003561,0.001968,0.003097,2.50075,l2,newton-cg,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.952381,0.952381,0.95,0.951905,0.030132,12
5,0.036580,0.007617,0.000429,0.000858,0.001,none,newton-cg,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.90,0.932381,0.024541,14
7,0.009762,0.003189,0.000000,0.000000,0.001,none,newton-cg,5.0005,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.809524,0.904762,0.904762,0.857143,0.95,0.885238,0.047913,19
8,0.007115,0.002357,0.004595,0.003855,0.001,none,newton-cg,7.50025,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.809524,0.857143,0.904762,0.857143,0.80,0.845714,0.037808,28
1,0.008373,0.000902,0.000371,0.000743,0.001,l2,newton-cg,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,37
0,0.012434,0.005596,0.004018,0.003555,0.001,l2,newton-cg,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.714286,0.666667,0.619048,0.761905,0.85,0.722381,0.079619,50


## param_grid_liblinear

In [196]:
param_grid_liblinear = {
    'LogisticRegression__tol': np.linspace(0.001, 10,5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['liblinear'],
    'LogisticRegression__penalty': ['l2', 'l1']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_liblinear , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'l1'],
                         'LogisticRegression__solver': ['liblinear'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [197]:
df_3 = pd.DataFrame(grid_search.cv_results_)
df_3.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_3 = df_3.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_3.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
25,0.007462,0.001790,0.000000,0.000000,5.0005,l1,liblinear,0.001,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.904762,0.904762,0.952381,0.95,0.942381,0.035520,1
15,0.001512,0.003024,0.006129,0.003170,2.50075,l1,liblinear,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.904762,0.904762,0.95,0.932857,0.037868,4
40,0.007904,0.001399,0.000000,0.000000,10.0,l2,liblinear,0.001,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.952381,0.904762,0.904762,0.904762,0.95,0.923333,0.022758,5
20,0.008258,0.000209,0.000000,0.000000,5.0005,l2,liblinear,0.001,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.904762,0.904762,0.857143,0.95,0.913810,0.035135,6
10,0.008348,0.000335,0.000511,0.001023,2.50075,l2,liblinear,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,0.904762,0.857143,0.95,0.904286,0.029370,8
26,0.008437,0.000473,0.000548,0.001097,5.0005,l1,liblinear,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.857143,0.857143,0.809524,0.761905,0.85,0.827143,0.037118,9
0,0.004930,0.004128,0.006139,0.002785,0.001,l2,liblinear,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,10
16,0.001008,0.002016,0.003997,0.003816,2.50075,l1,liblinear,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.809524,0.809524,0.761905,0.761905,0.85,0.798571,0.033388,13
11,0.001191,0.002382,0.006614,0.003317,2.50075,l2,liblinear,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.761905,0.857143,0.761905,0.761905,0.80,0.788571,0.037326,14
46,0.008017,0.000553,0.000043,0.000086,10.0,l1,liblinear,2.50075,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.714286,0.904762,0.714286,0.809524,0.80,0.788571,0.070861,14


## param_grid_sag

In [198]:

param_grid_sag = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['sag'],
    'LogisticRegression__penalty': ['l2', 'none']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_sag , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['sag'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [199]:
df_4 = pd.DataFrame(grid_search.cv_results_)
df_4.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_4 = df_4.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_4.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.007397,0.001913,0.003130,0.002581,2.50075,l2,sag,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
5,0.018873,0.002642,0.001715,0.002113,0.001,none,sag,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,5
41,0.006684,0.001444,0.003502,0.001092,10.0,l2,sag,2.50075,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.904762,0.952381,0.857143,0.95,0.913810,0.035135,10
31,0.007759,0.001168,0.000569,0.001035,7.50025,l2,sag,2.50075,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.904762,0.952381,0.952381,0.952381,0.80,0.912381,0.059140,11
17,0.006438,0.003234,0.001466,0.002564,2.50075,none,sag,5.0005,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,0.809524,0.904762,1.00,0.904762,0.060234,12
28,0.004433,0.003721,0.003673,0.004412,5.0005,none,sag,7.50025,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.857143,0.952381,0.857143,0.95,0.904286,0.042067,13
39,0.005116,0.003898,0.000000,0.000000,7.50025,none,sag,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.857143,1.000000,0.809524,0.90,0.903810,0.067370,14
22,0.002380,0.002260,0.005919,0.002169,5.0005,l2,sag,5.0005,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.809524,0.952381,0.904762,0.85,0.903333,0.068406,15
19,0.001650,0.002477,0.000000,0.000000,2.50075,none,sag,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.952381,0.904762,0.904762,0.952381,0.80,0.902857,0.055663,16
24,0.006553,0.001749,0.002231,0.001827,5.0005,l2,sag,10.0,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.857143,0.904762,0.857143,0.95,0.894762,0.034876,17


## param_grid_saga

In [200]:
param_grid_saga = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['saga'],
    'LogisticRegression__penalty': ['l1','l2', 'none'] 
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_saga , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l1', 'l2', 'none'],
                         'LogisticRegression__solver': ['saga'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [201]:
df_5 = pd.DataFrame(grid_search.cv_results_)
df_5.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_5 = df_5.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_5.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
15,0.014079,0.003304,0.002182,0.002950,2.50075,l1,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
31,0.011564,0.003406,0.006879,0.002382,5.0005,l1,saga,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.952381,1.000000,0.857143,1.00,0.942857,0.055533,9
10,0.020330,0.003431,0.002887,0.002439,0.001,none,saga,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,10
14,0.006352,0.003546,0.005318,0.003189,0.001,none,saga,10.0,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.904762,0.952381,0.904762,0.95,0.942381,0.035520,15
58,0.008762,0.000902,0.001604,0.003209,7.50025,none,saga,7.50025,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.904762,0.952381,0.904762,0.952381,0.95,0.932857,0.022956,16
49,0.010180,0.001545,0.004632,0.002716,7.50025,l1,saga,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.857143,0.952381,0.952381,0.95,0.932857,0.037868,16
42,0.012953,0.009078,0.004962,0.002988,5.0005,none,saga,5.0005,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.904762,0.952381,0.857143,0.95,0.932857,0.048384,18
32,0.011058,0.003706,0.006092,0.000965,5.0005,l1,saga,5.0005,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.904762,1.000000,0.761905,0.95,0.923333,0.088171,19
41,0.009022,0.004995,0.003193,0.001811,5.0005,none,saga,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.904762,1.000000,0.857143,0.90,0.922857,0.048971,20
66,0.014071,0.002486,0.002801,0.002516,10.0,l2,saga,2.50075,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.904762,0.904762,0.904762,0.95,0.913810,0.018095,22


In [202]:
param_grid_saga2 = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['saga'],
    'LogisticRegression__l1_ratio': np.linspace(0.001, 10, 1),
    'LogisticRegression__penalty': ['elasticnet']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_saga2 , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__l1_ratio': array([0.001]),
                         'LogisticRegression__penalty': ['elasticnet'],
                         'LogisticRegression__solver': ['saga'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [203]:
df_6 = pd.DataFrame(grid_search.cv_results_)
df_6.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_6 = df_6.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_6.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__l1_ratio,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.011775,0.003674,0.000504,0.001008,2.50075,0.001,elasticnet,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
21,0.014342,0.006867,0.004234,0.004345,10.0,0.001,elasticnet,saga,2.50075,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.857143,1.000000,0.952381,0.90,0.922857,0.048971,5
23,0.005549,0.003367,0.002579,0.003310,10.0,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.904762,0.857143,0.904762,0.95,0.904286,0.029370,6
14,0.009035,0.001626,0.005164,0.002810,5.0005,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.904762,1.000000,0.761905,0.95,0.904286,0.079403,6
9,0.004562,0.003209,0.001385,0.002770,2.50075,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.952381,0.904762,0.857143,0.90,0.903810,0.030177,8
24,0.008660,0.005362,0.002165,0.004329,10.0,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.904762,0.904762,0.857143,0.90,0.894286,0.018663,9
16,0.007810,0.001136,0.002829,0.003099,7.50025,0.001,elasticnet,saga,2.50075,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.857143,0.904762,0.857143,0.90,0.894286,0.035431,9
18,0.008312,0.000391,0.000000,0.000000,7.50025,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.857143,0.857143,0.904762,0.857143,0.95,0.885238,0.037265,11
17,0.008007,0.002046,0.001847,0.002295,7.50025,0.001,elasticnet,saga,5.0005,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.857143,0.809524,0.857143,0.95,0.885238,0.056593,12
13,0.006737,0.002570,0.001750,0.001271,5.0005,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 5.000500000000001, '...",0.761905,0.904762,0.952381,0.857143,0.95,0.885238,0.070829,12


# gabung kan semua ambilyg top 5

In [204]:
best=pd.concat([df_1.head(),df_2.head(),df_3.head(),df_4.head(),df_5.head(),df_6.head()])
best1=best.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
best1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,param_LogisticRegression__l1_ratio
10,0.016382,0.002580,0.003334,0.001484,2.50075,l2,lbfgs,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,NaN
6,0.011030,0.003367,0.000684,0.001368,0.001,none,newton-cg,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,NaN
10,0.007397,0.001913,0.003130,0.002581,2.50075,l2,sag,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,NaN
15,0.014079,0.003304,0.002182,0.002950,2.50075,l1,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,NaN
5,0.011775,0.003674,0.000504,0.001008,2.50075,elasticnet,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,0.001
21,0.009607,0.002523,0.001350,0.001758,5.0005,l2,lbfgs,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.952381,0.952381,0.904762,0.95,0.951905,0.030132,5,NaN
11,0.009684,0.003561,0.001968,0.003097,2.50075,l2,newton-cg,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.952381,0.952381,0.95,0.951905,0.030132,12,NaN
31,0.011564,0.003406,0.006879,0.002382,5.0005,l1,saga,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.952381,1.000000,0.857143,1.00,0.942857,0.055533,9,NaN
5,0.018873,0.002642,0.001715,0.002113,0.001,none,sag,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,5,NaN
10,0.020330,0.003431,0.002887,0.002439,0.001,none,saga,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,10,NaN


In [205]:
prin gax

SyntaxError: invalid syntax (1835790917.py, line 1)

In [206]:
data= ambil_best(best,10)
data

array([{'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['lbfgs'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [2.50075], 'LogisticRegression__l1_ratio': [0.001], 'LogisticRegression__penalty': ['elasticnet'], 'LogisticRegression__solver': ['saga'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [0.001], 'LogisticRegression__penalty': ['none'], 'LogisticRegression__solver': ['newton-cg'], 'LogisticRegression__tol': [2.50075]},
       {'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['l1'], 'LogisticRegression__solver': ['saga'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['sag'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [5.000500000000001], 'LogisticRegression__penalty': ['l1'], 'LogisticRegression__solver': ['liblinear'], 'Logis

In [207]:
if __name__ == "__main__":

    experiment_id = mlflow.create_experiment(
        name="project_logistic_iris_new",
        tags={"env": "dev", "version": "1.0.0"},
    )

    print(experiment_id)
    

mlflow.end_run()


652394146977343649


In [208]:
run_name = "log___"
increment_number = 1

In [209]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, recall_score

In [210]:
pipeline

Pipeline(steps=[('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(max_iter=10000))])

In [211]:

increment_number=input_mlflow(data,increment_number,X_train, y_train,pipeline,run_name,experiment_id)

run_id: 2cf29388fd1942509f33c7dc2539d8b3
run_id: 3f156b0081ab4250913318346e23574b
run_id: 4090b31e592743ce8d66ba56b3e820d4
run_id: 86cc3db5bba1442a89e1e6fddc200d4d
run_id: f5e3990be37e45ffb1224fb4177dc5be
run_id: 0bceeadfc7654420826ae5a03975a06c
run_id: 7ec89b3781d64416a6b9cf6420a05eb1
run_id: 8683f90946234ae0b39f0fab5694fc8c
run_id: 0838f7685d3f4e528c49aa81f6244125
run_id: 2aa8acf1afa949b882b2778f60667240
selesai


In [215]:
model_paths = [
  
    'mlruns/652394146977343649/2cf29388fd1942509f33c7dc2539d8b3/artifacts/grid_search__1',
    'mlruns/652394146977343649/4090b31e592743ce8d66ba56b3e820d4/artifacts/grid_search__3',
    'mlruns/652394146977343649/86cc3db5bba1442a89e1e6fddc200d4d/artifacts/grid_search__4',
    
    
    ]

test(model_paths,X_test,y_test)



================================| model 1 |==========================================
Classification Report for model _1:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.93      0.97        15
   virginica       0.94      1.00      0.97        17

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

Confusion Matrix for model _1:
 [[13  0  0]
 [ 0 14  1]
 [ 0  0 17]] 




================================| model 2 |==========================================
Classification Report for model _3:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      1.00      1.00        15
   virginica       1.00      1.00      1.00        17

    accuracy                           1.00        45
   macro avg       1.00    